In [2]:
using Revise

using BilevelTrajOpt
using ForwardDiff
using DiffResults

┌ Info: Precompiling BilevelTrajOpt [9688c538-179f-11e9-3174-495cea6b7f67]
└ @ Base loading.jl:1187


In [3]:
# Problem taken from:
# Sinha, Ankur, Pekka Malo, and Kalyanmoy Deb. 
# "A review on bilevel optimization: from classical to evolutionary approaches and applications." 
# IEEE Transactions on Evolutionary Computation 22.2 (2018): 276-295.

α = 10.
β = .2

δl = 1.
δf = 2.
γl = .3
γf = 5.8
cl = 10.
cf = 1.

P = (ql,qf) -> α - β*(ql + qf)
Cl = ql -> δl*ql*ql + γl*ql + cl
Cf = qf -> δf*qf*qf + γf*qf + cf

#13 (generic function with 1 method)

In [4]:
# closed form solution

qlopt = (2. * (β + δf)*(α - γl)-β * (α - γf))/(4. * (β + δf)*(β + δl) - 2. * β^2)
qfopt = (α - γf)/(2. * (β + δf)) - (β * (α - γl) - (β^2 * (α - γf))/(2. * (β + δf)))/(4. * (β + δf) * (β + δl) - 2. * β^2)

display(qlopt)
display(qfopt)

3.99236641221374

0.7730742539902844

In [5]:
# solution using our bilevel solver

function F(ql)
    fqf = qf -> -(P(ql[1],qf[1])*qf[1] - Cf(qf[1]))
    hqf = qf -> [0.]
    gqf = qf -> -qf
    qf0 = [0.]
    λ0 = [0.]
    μ0 = [0.]
    qfsol,λsol,μsol,Lsol = auglag_solve(qf0,λ0,μ0,fqf,hqf,gqf,inplace=false)
    
    J = -(P(ql[1],qfsol[1])*ql[1] - Cl(ql[1]))
    c = -ql
    
    vcat(J,c)
end

ql0 = [0.]
Fres = DiffResults.JacobianResult(F(ql0),ql0)
num_g = 1
function bilevel_prob(ql)
    ForwardDiff.jacobian!(Fres, F, ql)
    Fv = DiffResults.value(Fres)
    FJ = DiffResults.jacobian(Fres)
        
    J = Fv[1]
    c = Fv[2:1+num_g]

    gJ = FJ[1,:]
    gc = FJ[2:1+num_g,:]
    
    fail = false
    
    J, c, gJ, gc, fail
end

lb = [-1e19]
ub = [1e19]
options = Dict{String, Any}()
options["Derivative option"] = 1
options["Verify level"] = -1
options["Major optimality tolerance"] = 1e-3

qlsol, fsol, info = snopt(bilevel_prob, ql0, lb, ub, options)

display(info)
display(qlsol)

function compute_qfsol(ql)
    fqf = qf -> -(P(ql[1],qf[1])*qf[1] - Cf(qf[1]))
    hqf = qf -> [0.]
    gqf = qf -> -qf
    qf0 = [0.]
    λ0 = [0.]
    μ0 = [0.]
    qfsol,λsol,μsol,Lsol = auglag_solve(qf0,λ0,μ0,fqf,hqf,gqf)
end

qfsol,λsol,μsol,Lsol = compute_qfsol(qlsol)
display(qfsol)

"Finished successfully: optimality conditions satisfied"

1-element Array{Float64,1}:
 3.9923664122137463

1-element Array{Float64,1}:
 0.7730742539903015